In [13]:
import numpy as np
import pandas as pd
from Bio.Seq import Seq

def ddG(Kd, ref_Kd, R=1.9872036e-3, T=295):
    '''
    compute ∆∆G in kcal/mol relative to reference Kd
    '''
    return R*T*np.log(Kd/ref_Kd)

kBT = 0.593

In [2]:
# load in library of sequences to predict binding on
lib1_seqs = pd.read_csv('library1_names_sequences.csv')
lib2_seqs = pd.read_csv('library2_names_sequences.csv')

# PSAM prediction

In [3]:
def generate_matrix(seq):
    '''
    helper function to generate one-hot encoded sequence matrix
    '''
    seq_matrix = np.zeros((4, len(seq)))
    for j in range(len(seq)):
        if seq[j] == 'A':
            seq_matrix[0,j] = 1
        elif seq[j] == 'C':
            seq_matrix[1,j] = 1
        elif seq[j] == 'G':
            seq_matrix[2,j] = 1
        elif seq[j] == 'T':
            seq_matrix[3,j] = 1
    return seq_matrix

def get_PSAM_score(sequence, score_matrix):
    '''
    calculate PSAM score (a relative binding affinity) given a sequence and PSAM
    '''
    score = 0
    score_len = score_matrix.shape[0]
    # iterate through all positions of the sequence with a sliding window
    for j in range(len(sequence) - score_len + 1):
        seq_matrix = generate_matrix(sequence[j:j+score_len])
        diagonal = np.diagonal(np.matmul(score_matrix,seq_matrix))
        score += np.prod(diagonal)
    return score

In [4]:
# load in PSAM from Maerkl & Quake, Science, 2007.
MAX_PSAM = np.array(pd.read_csv('MAX_PSAM_MaerklQuake2007.csv',index_col=0)).T
# compute predicted binding
lib1_seqs['MAX_PSAM_pred'] = lib1_seqs['Sequence'].apply(lambda x: ddG(1,get_PSAM_score(x,MAX_PSAM)))
lib2_seqs['MAX_PSAM_pred'] = lib2_seqs['Sequence'].apply(lambda x: ddG(1,get_PSAM_score(x,MAX_PSAM)))

# Sliding Z-score prediction

In [5]:
def sliding_Z(sequence, Z_dict, k=8):
    # compute Z score for each window of length k in sequence
    sliding_Z_score = [Z_dict[sequence[i:i+k]] for i in range(len(sequence)-k+1)]
    # return the sum of Z scores
    return np.sum(sliding_Z_score)

In [6]:
# import Z scores
all_Zscores = pd.read_csv('Zscores_MAX_Pho4.csv').set_index('8mer')
# make dictionary from Z scores
MAX_Zscores = all_Zscores['MAX'].to_dict()
# add reverse complements to dictionary for completeness
MAX_Zscores.update({str(Seq(k).reverse_complement()): v for k,v in MAX_Zscores.items()})
# predicting binding
lib1_seqs['MAX_slidingZ_pred'] = lib1_seqs['Sequence'].apply(lambda x: sliding_Z(x,MAX_Zscores))
lib2_seqs['MAX_slidingZ_pred'] = lib2_seqs['Sequence'].apply(lambda x: sliding_Z(x,MAX_Zscores))

# Partition function prediction

In [7]:
def compute_gibbs_calibrated(E_array, slope, kBT=0.593):
    partition = [np.exp(-E/kBT) for E in E_array]
    return -slope*kBT*np.log(np.sum(partition))

def compute_enthalpy_calibrated(E_array, slope, kBT=0.593):
    enthalpy_array = [E*np.exp(-E/kBT) for E in E_array]
    enthalpy_array /= np.sum([np.exp(-E/kBT) for E in E_array])
    return slope*np.sum(enthalpy_array)

def compute_entropy_calibrated(E_array, slope, kBT=0.593):
    p_array = compute_p_array(E_array)
    entropy_array = [p*np.log(p) for p in p_array]
    return -slope*np.sum(entropy_array)

def gibbs_wrapper(sequence, E_dict, slope):
    return compute_gibbs_calibrated([E_dict[sequence[i:i+8]] for i in range(len(sequence)-7)],slope)

In [8]:
# see figure 2 in main text for calculation
slope_Pho4 = 3.00
slope_MAX = 4.03
# we are going to normalize our predictions to sequences with a motif surrounded by random flanks. they are listed below
ref_oligos_lib1 = [14,18,22]
ref_oligos_lib2 = [0,5]

In [9]:
# load in uPBM data from Badis et al. Science 2009.
MAX_rep1 = pd.read_csv('uPBM_data/MAX_8mers_rep1.txt',
                       delimiter='\t', names=['8mer', 'RC', 'intensity', 'Escore'], usecols=np.arange(4))
MAX_rep1['E'] = -np.log(MAX_rep1['intensity'])
MAX_rep1['E'] -= np.mean(MAX_rep1['E'])

MAX_rep2 = pd.read_csv('uPBM_data/MAX_8mers_rep2.txt',
                       delimiter='\t', names=['8mer', 'RC', 'intensity', 'Escore'], usecols=np.arange(4))
MAX_rep2['E'] = -np.log(MAX_rep2['intensity'])
MAX_rep2['E'] -= np.mean(MAX_rep2['E'])

# convert E scores to dictionary for compatibility with analysis
MAX_E_dict = {k:v for k,v in zip(MAX_rep1['8mer'], np.mean([MAX_rep1['E'], MAX_rep2['E']],axis=0))}
MAX_E_dict.update({k:v for k,v in zip(MAX_rep1['RC'], np.mean([MAX_rep1['E'], MAX_rep2['E']],axis=0))})

In [10]:
# predict binding
lib1_seqs['MAX_statmech_pred'] = lib1_seqs['Sequence'].apply(lambda x: gibbs_wrapper(x,MAX_E_dict,slope_MAX))
# normalize to reference sequence
lib1_seqs['MAX_statmech_pred'] -= lib1_seqs.loc[ref_oligos_lib1,'MAX_statmech_pred'].mean()

In [11]:
# predict binding
lib2_seqs['MAX_statmech_pred'] = lib2_seqs['Sequence'].apply(lambda x: gibbs_wrapper(x,MAX_E_dict,slope_MAX))
# normalize to reference sequence
lib2_seqs['MAX_statmech_pred'] -= lib2_seqs.loc[ref_oligos_lib2,'MAX_statmech_pred'].mean()

#### save predictions to file in csv format

In [12]:
lib1_seqs.to_csv('lib1_bindingpredictions.csv', index=False)
lib2_seqs.to_csv('lib2_bindingpredictions.csv', index=False)